加载分词器和模型，全部在Hugging Face Hub下载

In [2]:
import os
from model import Kronos, KronosTokenizer, KronosPredictor
# Load from Hugging Face Hub
tokenizer = KronosTokenizer.from_pretrained("NeoQuasar/Kronos-Tokenizer-base")
model = Kronos.from_pretrained("NeoQuasar/Kronos-base")

实例化预测器

In [3]:
# Initialize the predictor
predictor = KronosPredictor(model, tokenizer, device="cuda:0", max_context=512)

单一资产序列预测

In [4]:
import pandas as pd

# Load your data
df = pd.read_csv("./data/XSHG_5min_600977.csv")
df['timestamps'] = pd.to_datetime(df['timestamps'])

# Define context window and prediction length
lookback = 400
pred_len = 120

# Prepare inputs for the predictor
x_df = df.loc[:lookback-1, ['open', 'high', 'low', 'close', 'volume', 'amount']]
x_timestamp = df.loc[:lookback-1, 'timestamps']
y_timestamp = df.loc[lookback:lookback+pred_len-1, 'timestamps']

# Generate predictions
pred_df = predictor.predict(
    df=x_df,
    x_timestamp=x_timestamp,
    y_timestamp=y_timestamp,
    pred_len=pred_len,
    T=1.0,          # Temperature for sampling
    top_p=0.9,      # Nucleus sampling probability
    sample_count=1  # Number of forecast paths to generate and average
)

print("Forecasted Data Head:")
print(pred_df.head())

100%|██████████| 120/120 [00:04<00:00, 24.63it/s]

Forecasted Data Head:
                          open       high        low      close      volume  \
timestamps                                                                    
2024-06-28 14:05:00  10.822159  10.834056  10.811458  10.820602  523.151489   
2024-06-28 14:10:00  10.821757  10.831979  10.808309  10.816621  909.177246   
2024-06-28 14:15:00  10.819422  10.841615  10.815961  10.835768  584.458801   
2024-06-28 14:20:00  10.830839  10.838158  10.814431  10.819909  693.161621   
2024-06-28 14:25:00  10.815665  10.823578  10.783878  10.790836  766.734985   

                          amount  
timestamps                        
2024-06-28 14:05:00  563868.0625  
2024-06-28 14:10:00  975899.6875  
2024-06-28 14:15:00  628887.4375  
2024-06-28 14:20:00  746929.3750  
2024-06-28 14:25:00  837830.0000  


多资产序列预测

In [5]:
# Prepare multiple datasets for batch prediction
df_list = [df1, df2, df3]  # List of DataFrames
x_timestamp_list = [x_ts1, x_ts2, x_ts3]  # List of historical timestamps
y_timestamp_list = [y_ts1, y_ts2, y_ts3]  # List of future timestamps

# Generate batch predictions
pred_df_list = predictor.predict_batch(
    df_list=df_list,
    x_timestamp_list=x_timestamp_list,
    y_timestamp_list=y_timestamp_list,
    pred_len=pred_len,
    T=1.0,
    top_p=0.9,
    sample_count=1,
    verbose=True
)

# pred_df_list contains prediction results in the same order as input
for i, pred_df in enumerate(pred_df_list):
    print(f"Predictions for series {i}:")
    print(pred_df.head())

NameError: name 'df1' is not defined